# CIS 194: Homework 1

## Validating Credit Card Numbers

In this section, you will implement the validation algorithm for
credit cards. It follows these steps:
- Double the value of every second digit beginning from the right.
That is, the last digit is unchanged; the second-to-last digit is doubled; the third-to-last digit is unchanged; and so on. For example,
[1,3,8,6] becomes [2,3,16,6].
- Add the digits of the doubled values and the undoubled digits from the original number. For example, [2,3,16,6] becomes
2+3+1+6+6 = 18.
- Calculate the remainder when the sum is divided by 10. For the
above example, the remainder would be 8.

If the result equals 0, then the number is valid.

In [1]:
toDigitsRev :: Integer -> [Integer]
toDigitsRev x
  | x > 0 = x `mod` 10 : toDigitsRev (x `div` 10)
  | otherwise = []

toDigits :: Integer -> [Integer]
toDigits = reverse . toDigitsRev

doubleEveryOther :: [Integer] -> [Integer]
doubleEveryOther = reverse . zipWith (*) (cycle [1, 2]) . reverse

sumDigits :: [Integer] -> Integer
sumDigits = sum . concatMap toDigits

validate :: Integer -> Bool
validate = (== 0) . (`mod` 10) . sumDigits . doubleEveryOther . toDigits

In [2]:
validate 4012888888881881

True

In [3]:
validate 4012888888881882

False

## The Towers of Hanoi
Given the number of discs and names for the three pegs, hanoi
should return a list of moves to be performed to move the stack of
discs from the first peg to the second.

In [4]:
type Peg = String

type Move = (Peg, Peg)

hanoi :: Int -> Peg -> Peg -> Peg -> [Move]
hanoi 1 a b c = [(a, b)]
hanoi n a b c = hanoi (n - 1) a c b ++ (a, b) : hanoi (n - 1) c b a

In [5]:
ns = [1..3]
f x = hanoi x "a" "b" "c"
mapM_ (print . f) ns

[("a","b")]
[("a","c"),("a","b"),("c","b")]
[("a","b"),("a","c"),("b","c"),("a","b"),("c","a"),("c","b"),("a","b")]

### Optional
What if there are four pegs instead of three?
That is, the goal is still to move a stack of discs from the first peg to
the last peg, without ever placing a larger disc on top of a smaller
one, but now there are two extra pegs that can be used as “temporary” storage instead of only one. Write a function similar to hanoi
which solves this problem in as few moves as possible.

- 使用Frame-Stewart算法
- 需注意四柱汉诺塔的最优解法至今没有严格的数学证明，但Frame-Stewart算法被广泛认为是最优的

In [6]:
import Data.List (minimumBy)
import Data.Ord (comparing)

-- 计算最优的 k 值（分割点）
-- 使用动态规划思想找到使总步数最少的 k
optimalK :: Int -> Int
optimalK n = fst $ minimumBy (comparing snd) candidates
  where
    candidates = [(k, totalMoves n k) | k <- [1 .. n - 1]]
    totalMoves n k = 2 * t4 k + t3 (n - k)
    -- 四柱汉诺塔的步数（递归）
    t4 1 = 1
    t4 m = minimum [2 * t4 i + t3 (m - i) | i <- [1 .. m - 1]]
    -- 三柱汉诺塔的步数
    t3 m = 2 ^ m - 1

In [7]:
hanoi4 :: Int -> Peg -> Peg -> Peg -> Peg -> [Move]
hanoi4 0 _ _ _ _ = []
hanoi4 1 source target _ _ = [(source, target)]
hanoi4 n source target aux1 aux2 =
  let k = optimalK n
      -- 步骤1: 将前 k 个盘子移到 aux1（使用所有四个柱子）
      moves1 = hanoi4 k source aux1 aux2 target
      -- 步骤2: 将剩余 n-k 个盘子移到 target（只用三个柱子）
      moves2 = hanoi (n - k) source target aux2
      -- 步骤3: 将 k 个盘子从 aux1 移到 target（使用所有四个柱子）
      moves3 = hanoi4 k aux1 target aux2 source
   in moves1 ++ moves2 ++ moves3

In [10]:
(==129) . length $ hanoi4 15 "a" "b" "c" "d"

True